<a href="https://colab.research.google.com/github/Romulomrs/MortesPalestina/blob/main/CleaningData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import pandas as pd
import numpy as np

In [23]:
df = pd.read_json('dados.json')

In [24]:
pd.set_option('display.width', 1000)  # Aumenta o limite horizontal
pd.set_option('display.max_columns', None)  # Permite mostrar todas as colunas

In [25]:
# Preenche valores ausentes somente nas colunas numéricas
num_cols = df.select_dtypes(include=[np.number]).columns
df[num_cols] = df[num_cols].fillna(0)

In [26]:
df.fillna(0, inplace = True)
df = df.drop(['ext_massacres_cum','ext_killed','ext_killed_cum',
                 'ext_killed_children_cum','ext_killed_women_cum','ext_injured','ext_injured_cum','ext_civdef_killed_cum',
                 'ext_med_killed_cum','ext_press_killed_cum',],axis = 1)


In [27]:
# Converte a data para datetime
df['report_date'] = pd.to_datetime(df['report_date'])

In [28]:
# Cria coluna: dias desde o início da guerra
inicio_guerra = pd.to_datetime('2023-10-06')
df['dias_desde_inicio'] = (df['report_date'] - inicio_guerra).dt.days

In [29]:
colunas = [
    'killed_cum',
    'injured_cum',
    'med_killed_cum',
    'press_killed_cum',
    'killed_children_cum',
    'killed_women_cum'
]

# Substitui 0 por NA e faz forward fill em cada coluna
df[colunas] = df[colunas].replace(0, pd.NA).ffill()

/tmp/ipython-input-29-2473132812.py:11: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[colunas] = df[colunas].replace(0, pd.NA).ffill()


In [30]:
# Filtra apenas os dados a partir de 7 de outubro de 2023
df = df[df['report_date'] >= inicio_guerra]
df['ano_mes'] = df['report_date'].dt.to_period('M')

In [31]:
print(df)

    report_date report_source  report_period  killed  killed_cum  injured_cum  med_killed_cum  press_killed_cum killed_children_cum killed_women_cum  injured  massacres_cum  civdef_killed_cum  killed_recovered  killed_truce_new  killed_succumbed  killed_committee  dias_desde_inicio  ano_mes
576  2025-05-05        mohtel             24    32.0     52567.0     118610.0          1402.0             212.0             18000.0          12400.0    119.0            0.0                0.0               9.0              23.0               0.0               0.0                577  2025-05
577  2025-05-06        mohtel             24    48.0     52615.0     118752.0          1402.0             212.0             18000.0          12400.0    142.0            0.0                0.0               0.0              48.0               0.0               0.0                578  2025-05
578  2025-05-07        mohtel             24    38.0     52653.0     118897.0          1402.0             214.0             